# 第16章 运算符重载

对运算符重载的限制：
- 不能改变内置类型的运算符表达的意思。
- 不能新建运算符，只能重载现有运算符。
- 有些运算符不能重载：`is`、`and`、`or`和`not`。

## 16.1 一元运算符

- 一元取反算术运算符（`-`）：由`__neg__`实现。
- 一元取正算术运算符（`+`）：由`__pos__`实现。
- 按位否定（取反）整数（`~`）：由`__invert__`实现。

In [16]:
from array import array
import reprlib
import math
import functools
import operator
import itertools


class VectorV6:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.hypot(*self)

    def __neg__(self):
        # 每个分量取反
        return self.__class__(-x for x in self)

    def __pos__(self):
        # 每个分量保持不变
        return self.__class__(self)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self)
            return cls(self._components[key])
        index = operator.index(key)
        return self._components[index]

    __match_args__ = ('x', 'y', 'z', 't')

    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name)
        except ValueError:
            pos = -1
        if 0 <= pos < len(self._components):
            return self._components[pos]
        msg = f'{cls.__name__!r} object has no attribute {name!r}'
        raise AttributeError(msg)

    def angle(self, n):
        r = math.hypot(*self[n:])
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

    def __add__(self, other):
        # 重载加法运算符
        try:
            # 生成(a,b)形式的元组
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return self.__class__(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented

    def __radd__(self, other):
        return self + other

In [17]:
v1 = VectorV6([3, 4, 5])
v1 + (10, 20, 30)

Vector([13.0, 24.0, 35.0])

如果中缀运算符方法抛出异常，那么它将终止运算符分派机制。

## 16.2 重载标量乘法运算符（`*`）和点乘运算符（`@`）

In [26]:
from collections import abc

class VectorV7(VectorV6):
    def __mul__(self, scalar):
        try:
            factor = float(scalar)
        except TypeError:
            return NotImplemented
        return self.__class__(n * factor for n in self)

    def __rmul__(self, scalar):
        return self * scalar

    def __matmul__(self, other):
        if (isinstance(other, abc.Sized) and
            isinstance(other, abc.Iterable)):
            if len(self) == len(other):
                return sum(a * b for a, b in zip(self, other))
            else:
                raise ValueError('@ requires vectors of equal length.')
        else:
            return NotImplemented

    def __rmatmul__(self, other):
        return self @ other

In [27]:
v1 = VectorV7([1, 2, 3])
v1 * 10

Vector([10.0, 20.0, 30.0])

In [28]:
11 * v1

Vector([11.0, 22.0, 33.0])

In [29]:
va = VectorV7([1, 2, 3])
vz = VectorV7([5, 6, 7])
va @ vz

38.0

In [30]:
[10, 20, 30] @ vz

380.0

## 16.3 算术运算符总结

| 运算符 | 正向方法 | 反向方法 | 就地方法 | 说明 |
| :---: | :---: | :---: | :---: | :---|
| + | \_\_add\_\_ | \_\_radd\_\_ | \_\_iadd\_\_ | 加法或拼接 |
| - | \_\_sub\_\_ | \_\_rsub\_\_ | \_\_isub\_\_ | 减法 |
| * | \_\_mul\_\_ | \_\_rmul\_\_ | \_\_imul\_\_ | 乘法或重复复制 |
| / | \_\_truediv\_\_ | \_\_rtruediv\_\_ | \_\_itruediv\_\_ | 除法 |
| // | \_\_floordiv\_\_ | \_\_rfloordiv\_\_ | \_\_ifloordiv\_\_ | 整除 |
| % | \_\_mod\_\_ | \_\_rmod\_\_ | \_\_imod\_\_ | 求模 |
| divmod() | \_\_divmod\_\_ | \_\_rdivmod\_\_ | \_\_idivmod\_\_ | 返回由整除的商和模数组成的元组 |
| \*\*、pow() | \_\_pow\_\_ | \_\_rpow\_\_ | \_\_ipow\_\_ | 求幂 |
| @ | \_\_matmul\_\_ | \_\_rmatmul\_\_ | \_\_imatmul\_\_ | 矩阵乘法 |
| & | \_\_and\_\_ | \_\_rand\_\_ | \_\_iand\_\_ | 位与 |
| | | \_\_or\_\_ | \_\_ror\_\_ | \_\_ior\_\_ | 位或 |
| ^ | \_\_xor\_\_ | \_\_rxor\_\_ | \_\_ixor\_\_ | 位异或 |
| << | \_\_lshift\_\_ | \_\_rlshift\_\_ | \_\_ilshift\_\_ | 按位左移 |
| >> | \_\_rshift\_\_ | \_\_rrshift\_\_ | \_\_irshift\_\_ | 按位右移 |

## 16.4 众多比较运算符

In [31]:
class VectorV8(VectorV7):
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return (len(self) == len(other) and
                    all(a == b for a, b in zip(self, other)))
        else:
            return NotImplemented

In [32]:
va = VectorV8([1.0, 2.0, 3.0])
vb = VectorV8(range(1, 4))
va == vb

True

In [33]:
t3 = (1, 2, 3)
va == t3

False

## 16.5 杂谈

- 运算符重载的优缺点讨论：
     - James Gosling（Java之父）：世界上有成千上万个运算符，只有少数几个适合重载。决定不让Java支持运算符重载。
     - Guido van Rossum（Python之父）：不放任用户随意创建像`<=>`或`:)`这样新的运算符，这样禁止了用户对运算符的异想天开，而且能让Python解析器保持简单。社区中约有10%的人能正确地使用和真正关心运算符重载。
     - 作者：重载的运算符，如果使用得当，的确能让代码更易于阅读和编写。